# g(x) = +1 if there are distinct i,j,k in {1,...,10} with x[i]+x[j]+x[k] = 5, else -1

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

    # Define the ground truth function g
def g_function(x):
      n = len(x)
      for i in range(n):
          for j in range(i + 1, n):
              for k in range(j + 1, n):
                  if x[i] + x[j] + x[k] == 5:
                      return 1
      return -1

def generate_dataset(num_samples, seed):
    np.random.seed(seed)
    # Generate random samples from {0, 1, ..., 9} for 10 components as question wanted
    data = np.random.randint(0, 10, size=(num_samples, 10))
    # Apply the function to each data point
    labels = np.array([g_function(x) for x in data])
    return data, labels

# Generate dataset and assign to global variables
data, labels = generate_dataset(num_samples=10000, seed=20)
print("Data:", data)
print("Labels:", labels)

Data: [[3 9 4 ... 6 8 5]
 [3 0 6 ... 7 5 2]
 [6 9 3 ... 2 3 1]
 ...
 [4 8 5 ... 2 5 9]
 [4 5 2 ... 4 7 0]
 [4 5 7 ... 4 8 9]]
Labels: [ 1  1  1 ... -1  1 -1]


# Compute Statistics

In [ ]:
def compute_statistics(labels):
    #Show classes statistics
    classes, counts = np.unique(labels, return_counts=True)
    stats = dict(zip(classes, counts))
    return stats

# Compute dataset statistics
statistics = compute_statistics(labels)
print("Dataset class balance:", statistics)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=20)

print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])
print("Training set class balance:", compute_statistics(y_train))
print("Test set class balance:", compute_statistics(y_test))



Dataset class balance: {-1: 3540, 1: 6460}
Training set size: 8000
Test set size: 2000
Training set class balance: {-1: 2853, 1: 5147}
Test set class balance: {-1: 687, 1: 1313}


# Implementing permutaion sensitive NN

##Model of NN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define a neural network
#Helpful link that I used: https://www.youtube.com/watch?v=kY14KfZQ1TI&t=8s
class PermutationSensitiveNN(nn.Module):
    def __init__(self, input_dim, hidden_dim,hidden_dim2, output_dim):
        super(PermutationSensitiveNN, self).__init__()
        torch.manual_seed(12345)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x


#Prepare data for pytorch

In [ ]:
# Convert labels from {-1, +1} to {0, 1}
y_train = (y_train + 1) // 2
y_test = (y_test + 1) // 2

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

##Initialize Parameters

In [ ]:
model = PermutationSensitiveNN(input_dim=10, hidden_dim=128, hidden_dim2=64, output_dim=1)
criterion = nn.BCEWithLogitsLoss()  # Binary classification loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

##Training and Evaluation

In [ ]:
for epoch in range(20):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0).long()  # Convert logits to {0, 1}
            total += labels.size(0)
            correct += (predicted == labels.long()).sum().item()

    test_accuracy = correct / total
    print(f"Epoch [{epoch+1}/20], Loss: {avg_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch [1/20], Loss: 0.4827, Test Accuracy: 0.8265
Epoch [2/20], Loss: 0.4073, Test Accuracy: 0.8170
Epoch [3/20], Loss: 0.3912, Test Accuracy: 0.8390
Epoch [4/20], Loss: 0.3812, Test Accuracy: 0.8405
Epoch [5/20], Loss: 0.3708, Test Accuracy: 0.8280
Epoch [6/20], Loss: 0.3697, Test Accuracy: 0.8385
Epoch [7/20], Loss: 0.3638, Test Accuracy: 0.8495
Epoch [8/20], Loss: 0.3599, Test Accuracy: 0.8450
Epoch [9/20], Loss: 0.3510, Test Accuracy: 0.8465
Epoch [10/20], Loss: 0.3547, Test Accuracy: 0.8440
Epoch [11/20], Loss: 0.3463, Test Accuracy: 0.8390
Epoch [12/20], Loss: 0.3451, Test Accuracy: 0.8380
Epoch [13/20], Loss: 0.3399, Test Accuracy: 0.8440
Epoch [14/20], Loss: 0.3389, Test Accuracy: 0.8365
Epoch [15/20], Loss: 0.3324, Test Accuracy: 0.8410
Epoch [16/20], Loss: 0.3347, Test Accuracy: 0.8415
Epoch [17/20], Loss: 0.3337, Test Accuracy: 0.8465
Epoch [18/20], Loss: 0.3300, Test Accuracy: 0.8425
Epoch [19/20], Loss: 0.3270, Test Accuracy: 0.8550
Epoch [20/20], Loss: 0.3214, Test Accura

In [ ]:
# Now we want to test sensitivity of model
# Example input
original_input = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 0]], dtype=torch.float32)

# Shuffle the input
shuffled_input = original_input[:, torch.randperm(original_input.size(1))]

# Model output for original and shuffled inputs
model.eval()
original_output = model(original_input)
shuffled_output = model(shuffled_input)

print("Original Input:", original_input)
print("Shuffled Input:", shuffled_input)
print("Original Output:", original_output.item())
print("Shuffled Output:", shuffled_output.item())


Original Input: tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9., 0.]])
Shuffled Input: tensor([[9., 1., 3., 6., 0., 4., 7., 8., 2., 5.]])
Original Output: 2.379833698272705
Shuffled Output: 1.1561832427978516


# Implementing of permutation invariant NN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#Deep Sets model
#Helpful link that I used: https://jduarte.physics.ucsd.edu/iaifi-summer-school/1.3_deep_sets.html
class DeepSets(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepSets, self).__init__()
        torch.manual_seed(12345)

        #phi, Processes each element of the input individually
        self.phi = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # rho, Processes the pooled output
        self.rho = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        # Applying phi to each component, then sum operation and rho
        phi_output = self.phi(x)

        pooled_output = torch.sum(phi_output, dim=1)

        output = self.rho(pooled_output)
        return output

# Convert labels from {-1, +1} to {0, 1}
y_train = (y_train + 1) // 2
y_test = (y_test + 1) // 2

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

#Deep Sets model
model = DeepSets(input_dim=1, hidden_dim=128, output_dim=1)
criterion = nn.BCEWithLogitsLoss()  # Binary classification loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Reshape inputs to process each element individually
        inputs = inputs.view(-1, 10, 1)
        outputs = model(inputs)

        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # Evaluate
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.view(-1, 10, 1)
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0).long()  # Convert logits to {0, 1}
            total += labels.size(0)
            correct += (predicted == labels.long()).sum().item()

    test_accuracy = correct / total
    print(f"Epoch [{epoch+1}/20], Loss: {avg_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch [1/20], Loss: 0.4665, Test Accuracy: 0.8430
Epoch [2/20], Loss: 0.3693, Test Accuracy: 0.8640
Epoch [3/20], Loss: 0.3536, Test Accuracy: 0.8605
Epoch [4/20], Loss: 0.3369, Test Accuracy: 0.8720
Epoch [5/20], Loss: 0.3319, Test Accuracy: 0.8465
Epoch [6/20], Loss: 0.3133, Test Accuracy: 0.8795
Epoch [7/20], Loss: 0.3020, Test Accuracy: 0.8905
Epoch [8/20], Loss: 0.2829, Test Accuracy: 0.8960
Epoch [9/20], Loss: 0.2791, Test Accuracy: 0.8965
Epoch [10/20], Loss: 0.2782, Test Accuracy: 0.8970
Epoch [11/20], Loss: 0.2734, Test Accuracy: 0.8990
Epoch [12/20], Loss: 0.2681, Test Accuracy: 0.8940
Epoch [13/20], Loss: 0.2671, Test Accuracy: 0.8965
Epoch [14/20], Loss: 0.2657, Test Accuracy: 0.8970
Epoch [15/20], Loss: 0.2644, Test Accuracy: 0.8995
Epoch [16/20], Loss: 0.2574, Test Accuracy: 0.9045
Epoch [17/20], Loss: 0.2556, Test Accuracy: 0.9055
Epoch [18/20], Loss: 0.2576, Test Accuracy: 0.9030
Epoch [19/20], Loss: 0.2435, Test Accuracy: 0.9095
Epoch [20/20], Loss: 0.2401, Test Accura

In [ ]:
# Test for permutation-invariance
original_input = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 0]], dtype=torch.float32).view(1, 10, 1)
shuffled_input = original_input[:, torch.randperm(original_input.size(1)), :]

model.eval()
original_output = model(original_input)
shuffled_output = model(shuffled_input)

print("Original Input:", original_input)
print("Shuffled Input:", shuffled_input)
print("Original Output:", original_output.item())
print("Shuffled Output:", shuffled_output.item())


Original Input: tensor([[[1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [7.],
         [8.],
         [9.],
         [0.]]])
Shuffled Input: tensor([[[8.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [9.],
         [1.],
         [0.],
         [7.]]])
Original Output: -12.324579238891602
Shuffled Output: -12.324580192565918
